In [1]:
# This is importent when we want to call this as a python script, because jupyter naturally has a higher recursion depth
import sys
sys.setrecursionlimit(3000)

# Print the PID when using nohup
import os
from icecream import ic
ic(os.getpid())

ic| os.getpid(): 835405


835405

In [2]:
from gymportal.data.ev_generators import get_standard_generator, RealWorldGenerator
from acnportal.acnsim import Linear2StageBattery
from gymportal.data.battery_generators import CustomizableBatteryGenerator
from gymportal.sim import get_charging_network, Recomputer, EvaluationSimulator, SimGenerator
from datetime import datetime, timedelta

import pytz
timezone = pytz.timezone("America/Los_Angeles")


charging_network = get_charging_network('simple_acn', basic_evse=True, voltage=208,
                                        network_kwargs={
                                            'station_ids': ['CA-504', 'CA-503', 'CA-502', 'CA-501'],
                                             #'station_ids': ['CA-501'],
"aggregate_cap": 32 * 208 / 1000})

# charging_network = get_charging_network('caltech', basic_evse=True, voltage=208,
#                                         network_kwargs={"transformer_cap": 150})

battery_generator = CustomizableBatteryGenerator(
    voltage=208,
    period=1,
    battery_types=[
        Linear2StageBattery],
    max_power_function="normal",
)

ev_generator = RealWorldGenerator(battery_generator=battery_generator, site='caltech', period=1)
# ev_generator = get_standard_generator(
#     'caltech', battery_generator, seed=42, frequency_multiplicator=frequency_multiplicator, duration_multiplicator=2)

train_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7,
    n_intervals=46,
    start_date=timezone.localize(datetime(2019, 1, 1)),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(train_generator.end_date + timedelta(days=1))

eval_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=7,
    n_intervals=1,
    start_date=train_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(eval_generator.end_date + timedelta(days=1))

validation_generator = SimGenerator(
    charging_network=charging_network,
    simulation_days=14,
    n_intervals=1,
    start_date=eval_generator.end_date + timedelta(days=1),
    ev_generator=ev_generator,
    recomputer=Recomputer(recompute_interval=10, sparse=True),
    sim_class=EvaluationSimulator,
)

ic(validation_generator.end_date + timedelta(days=1))
pass

ic| train_generator.end_date + timedelta(days=1): datetime.datetime(2019, 11, 20, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| eval_generator.end_date + timedelta(days=1): datetime.datetime(2019, 11, 28, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)
ic| validation_generator.end_date + timedelta(days=1): datetime.datetime(2019, 12, 13, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>)


In [3]:
from src.pv.pv import read_pv_data

df_pv = read_pv_data("../pv_150kW.csv")
df_pv.describe()

,P,G(i),H_sun,T2m,WS10m,Int
count,140256.000000,140256.000000,140256.000000,140256.000000,140256.000000,140256.0
mean,30395.798319,267.983145,16.748906,17.931872,1.654366,0.0
std,40648.543625,356.157287,22.166668,7.394042,0.867817,0.0
min,0.000000,0.000000,-5.880000,-2.230000,0.000000,0.0
25%,0.000000,0.000000,0.000000,12.360000,0.970000,0.0
50%,87.000000,7.260000,0.520000,17.310000,1.590000,0.0
75%,65764.500000,564.280000,31.982500,23.010000,2.280000,0.0
max,132601.500000,1143.220000,78.090000,46.460000,7.930000,0.0


In [4]:
from gymportal.environment import *
from src.pv.observations import pv_observation_mean
from src.pv.rewards import *

observation_objects = [
    charging_rates_observation_normalized(),
    percentage_of_magnitude_observation(),
    diff_pilots_charging_rates_observation_normalized(),
    cyclical_minute_observation(),
    cyclical_day_observation(),
    cyclical_month_observation(),
    cyclical_minute_observation_stay(),
    energy_delivered_observation_normalized(),
    num_active_stations_observation_normalized(),
    pilot_signals_observation_normalized(),
    cyclical_minute_observation_arrival(),
    cyclical_day_observation_arrival(),
    cyclical_month_observation_arrival(),
    pv_observation_mean(df_pv),
]

reward_objects = [
    pv_utilization_reward(df_pv),
    grid_use_penalty(df_pv),
    unused_pv_penalty(df_pv),
    charging_reward(),
    soft_charging_reward_pv_weighted(df_pv, transformer_cap=150),
]

In [5]:
import dill as pickle

with open("../caltech_#stations=54_#days=7_#intervals=46_seed=8734956.pkl", "rb") as file:
    train_generator = pickle.load(file)

In [6]:
steps_per_epoch = 0
for sim in train_generator._sim_memory:
    steps_per_epoch += len(sim.event_queue.queue)

ic(steps_per_epoch)

ic| steps_per_epoch: 37656


37656

In [7]:
train_config = {"observation_objects": observation_objects, "action_object": zero_centered_single_charging_schedule_normalized(),
                "reward_objects": reward_objects,
                "simgenerator": train_generator,
                "meet_constraints": True}

eval_config = train_config | {'simgenerator': eval_generator}
validation_config = train_config | {'simgenerator': validation_generator}

In [8]:
steps_per_epoch = int(steps_per_epoch / 4)

In [9]:
from src.cleanRL import Args
from gymportal.evaluation import *
from src.pv.metrics import *

args = Args(
    exp_name="test on simple_acn",
    total_timesteps=steps_per_epoch * 1,
    num_steps=steps_per_epoch,
    num_envs=1,
    ent_coef=1e-4,
    # wandb:
    track=True,
    wandb_project_name="cleanRL_test",
    wandb_entity="tsturm-university-kassel",
    save_model=True,
    # my own stuff:
    train_config=train_config,
    eval_config=eval_config,
    eval_seed=930932,
    eval_metrics={
        "SoC >= 90%": percentage_soc,
        "mean SoC": mean_soc,
        "median SoC": median_soc,
        "prop feasible steps": proportion_of_feasible_charging,
        "prop feasible charge": proportion_of_feasible_charge,
        "pv utilization": lambda sim: pv_utilization_metric(sim, df_pv),
        "grid usage": lambda sim: grid_use_metric(sim, df_pv),
        "unused pv": lambda sim: unused_pv_metric(sim, df_pv),
    },
)

In [10]:
from src.cleanRL import train_ppo
from src.cleanRL.environment import make_env


train_ppo(args, make_env)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tsturm (tsturm-university-kassel). Use `wandb login --relogin` to force relogin


/home/guest/tsturm/acn-experiments/.conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/guest/tsturm/acn-experiments/.conda/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/home/guest/tsturm/acn-experiments/.conda/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


global_step=975, episodic_return=[17896454.]
global_step=2023, episodic_return=[12479009.]
global_step=3002, episodic_return=[28787880.]
global_step=3976, episodic_return=[16873542.]
global_step=4946, episodic_return=[27450388.]
global_step=5909, episodic_return=[11697692.]
global_step=6988, episodic_return=[33318292.]
global_step=7958, episodic_return=[16328238.]
global_step=8913, episodic_return=[17812410.]
SPS: 5


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x57 and 664x64)

In [ ]:
import torch


run_name = f"{args.exp_name}__{args.seed}"
path = f"runs/{run_name}/{args.exp_name}.cleanrl_model"

agent = torch.load(path)

In [ ]:
from src.cleanRL.scheduler import CleanRLSchedule
ppo_scheduler = CleanRLSchedule(agent)

In [ ]:
from gymportal.evaluation import ACNSchedule
from acnportal.algorithms import UncontrolledCharging, SortedSchedulingAlgo, last_come_first_served, \
    first_come_first_served


models = {
    "PPO": ppo_scheduler,
    "FCFS": ACNSchedule(SortedSchedulingAlgo(first_come_first_served)),
    "LCFS": ACNSchedule(SortedSchedulingAlgo(last_come_first_served)),
    "Uncontrolled": ACNSchedule(UncontrolledCharging()),
}

models

In [ ]:
from src.run_simulation import run_simulations


df = run_simulations(models, metrics=args.eval_metrics,
                    config=args.eval_config, seed=args.eval_config["simgenerator"].seed)

df

In [ ]:
import wandb


wandb.run